In [ ]:
# Install Kaggle CLI
!pip install kaggle


!mkdir -p ~/.kaggle


from google.colab import files
files.upload()


!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Download the dataset from Kaggle
!kaggle datasets download -d ananthu017/emotion-detection-fer

# Unzip the dataset
!unzip emotion-detection-fer.zip -d Emotion


# STEP 1: Import Required Libraries


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2

#STEP 2: Set Dataset Directory Paths


In [ ]:
base_dir = "Emotion"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

#STEP 3: Data Preprocessing & Augmentation

In [ ]:
img_width, img_height = 48, 48
batch_size = 64

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Flow training images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical"
)

# Flow test/validation images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical"
)

# Emotion labels
emotion_labels = list(train_generator.class_indices.keys())
print("Emotion Labels:", emotion_labels)

#STEP 4: Build CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(emotion_labels), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

#STEP 5: Train the Model


In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_emotion_model.h5", save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=25,
    validation_data=test_generator,
    callbacks=callbacks
)


#STEP 6: Plot Accuracy & Loss

In [ ]:
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', color='red')
plt.plot(history.history['val_loss'], label='Validation Loss', color='green')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


# STEP 7: Save Model

In [ ]:
model.save("emotion_model_final.h5")
print(" Model saved as emotion_model_final.h5")

# STEP 8: Predict Emotion from Uploaded Image

In [ ]:
def predict_emotion(image_path):
    """
    Predicts emotion from an image file.
    """
    img = load_img(image_path, target_size=(48, 48), color_mode="grayscale")
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    model = load_model("emotion_model_final.h5")
    predictions = model.predict(img_array)
    predicted_label = emotion_labels[np.argmax(predictions)]

    print(f"Predicted Emotion: {predicted_label}")
    return predicted_label

# STEP 9:  Upload Image for Prediction

In [ ]:
from google.colab import files

print("\nPlease upload an image to test emotion prediction:")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nPredicting emotion for: {filename}")
    emotion = predict_emotion(filename)
    print(f" Detected Emotion: {emotion}")